# Testing the size of the aggregated Hypercube

Used CP-Nets

In [431]:
# list of names to use
files = ["cpnet_n5c4d2_0000.xml", "cpnet_n5c4d2_0001.xml"]
#files = ["cpnet_n6c5d2_0000.xml", "cpnet_n6c5d2_0001.xml"]

# get n value
n = files[0][7:9]
if not n.isdigit():
    n = n[0]
n = int(n)    
    
# should we check whether n value is the same for all files??          

In [432]:
"""
genrates a list of the used variables
"""
def get_variables(n):
    vars = []
    for i in range(1,n+1):
        vars.append("x" + str(i))
    return vars 

variables = get_variables(n)

Helper functions for generating a data structure of the XML files

This only functions for binary domain cpnets!! (because the data structure simplifies the preference)

In [433]:
"""
forms a string representing the preference, e.g. x1=2  (from x1 and 2:1) 
meaning x1=2 is preferred over x1=1
"""
def reform_pref_string(variable, preference):
    new_string = variable + "=" + preference.split(":")[0]
    return new_string

"""
Input - ordered dict of a CP-Net formed by xmltodict
Output - a dictionary representing the rules
"""
def generating_rules(cpnet_dict):
    dependencies = set()
    rules = {}
    for rule in cpnet_dict['PREFERENCE-SPECIFICATION']['PREFERENCE-STATEMENT']:
        variable_name = rule['PREFERENCE-VARIABLE'] #e.g. x1
        preference = rule['PREFERENCE'] #e.g. 2:1
        conditions = []
        if len(rule) > 3: # can be a list of conditions or a single item e.g. x2=2 or [x1=2, x2=2]
            conditions = rule['CONDITION']
            if type(rule['CONDITION']) is str:
                conditions = [conditions]
                
        cond_set = frozenset(conditions)
        pref = reform_pref_string(variable_name, preference)
        
        # add rule to the dictionary
        if cond_set in rules:
            rules[cond_set].append(pref)
        else:
            rules[cond_set] = [pref]
            
        # add dependency
        for cond in conditions:
            dep = (variable_name, cond[:2])
            dependencies.add(dep)
                    
    return rules, dependencies              

Reading XML files and generating a data structure which represent the rules of the CP-Net (a list of dictionaries)

In [434]:
import xmltodict

cp_list = []
dep_list = []
for file in files:
    with open("./CPNets/" + file) as fd:
        doc = xmltodict.parse(fd.read())
        cp, cp_dep = generating_rules(doc)
        cp_list.append(cp)
        dep_list.append(cp_dep)
        
        print(cp)       

{frozenset({'x3=1'}): ['x1=1'], frozenset({'x3=2'}): ['x1=2'], frozenset({'x3=1', 'x5=1', 'x4=1', 'x1=1'}): ['x2=2'], frozenset({'x3=1', 'x5=2', 'x4=1', 'x1=1'}): ['x2=2'], frozenset({'x3=1', 'x4=2', 'x5=1', 'x1=1'}): ['x2=2'], frozenset({'x3=1', 'x5=2', 'x4=2', 'x1=1'}): ['x2=1'], frozenset({'x5=1', 'x3=2', 'x4=1', 'x1=1'}): ['x2=2'], frozenset({'x5=2', 'x3=2', 'x4=1', 'x1=1'}): ['x2=1'], frozenset({'x3=2', 'x4=2', 'x5=1', 'x1=1'}): ['x2=1'], frozenset({'x5=2', 'x3=2', 'x4=2', 'x1=1'}): ['x2=1'], frozenset({'x5=1', 'x1=2', 'x4=1', 'x3=1'}): ['x2=1'], frozenset({'x5=2', 'x1=2', 'x4=1', 'x3=1'}): ['x2=2'], frozenset({'x4=2', 'x1=2', 'x5=1', 'x3=1'}): ['x2=1'], frozenset({'x5=2', 'x4=2', 'x1=2', 'x3=1'}): ['x2=2'], frozenset({'x5=1', 'x3=2', 'x1=2', 'x4=1'}): ['x2=2'], frozenset({'x3=2', 'x1=2', 'x4=1', 'x5=2'}): ['x2=2'], frozenset({'x3=2', 'x4=2', 'x1=2', 'x5=1'}): ['x2=1'], frozenset({'x3=2', 'x4=2', 'x1=2', 'x5=2'}): ['x2=1'], frozenset(): ['x3=1'], frozenset({'x3=1', 'x1=1'}): ['x4=

In [435]:
# get size of the cp-nets

# number of rules
count = 0 # elements in the dictionary have to be counted
for net in cp_list:
    count_net = 0
    for key in net:
        count_net += len(net[key])
    print(count_net)    
    count += count_net    
print(count)


# number of dependencies
print("Dependencies")
count = 0
for dep in dep_list:
    print(len(dep))
    count += len(dep)
print(count)

31
31
62
Dependencies
10
10
20


Aggregating the CP-Nets to one hypercube CP-Net

In [436]:
"""
Generator for tuples of neighbours in the hypercube graph. Returns only positive variables.
If not included, the variable is negative.
TODO: Works though, but generates things double...
"""
def neighbours(seq_orig):
    seq = seq_orig.copy()
    for item in range(len(seq)):
        seq.pop(item)    
        yield (seq_orig, seq)
        yield from neighbours(seq)
        seq = seq_orig.copy()

#seq = ["x1", "x2", "x3", "x4"]        
#for i  in neighbours(seq):
#    print(i)

In [437]:
"""
Helper function for formatting
Input: two neighbours as returned by neighbour function
Output: Tuple - first element the varying variable between the elements, second and third element set of constant variables (only differently formatted)
"""
def diff_neighbours(seq1, seq2, vars):
    ones = set(seq1).intersection(seq2)
    varying = set(seq1).difference(seq2)
    twos = set(vars).difference(seq1)
    return (varying.pop(), set([i+"=1" for i in ones]).union([i+"=2" for i in twos]))


In [438]:
"""
For searching for matching rules in the dictionary (for a pair of neighbours in the hypercube)
"""
def matching_preferences(dictionary, premises, variable):
    prefs = []
    for key in dictionary:
        if key <= premises:
            for pref in dictionary[key]:
                if pref.startswith(variable):
                    prefs.append(pref)
    return prefs        

In [439]:
"""
Applying majority rule to collected preferences
"""
def maj_rule(prefs):
    votes_ones = 0
    votes_twos = 0
    
    for pref in prefs:
        if pref.endswith("1"):
            votes_ones += 1
        elif pref.endswith("2"):
            votes_twos += 1
        else:
            print("Error: " + pref)
            
    if votes_ones > votes_twos:
        return "1"
    elif votes_ones < votes_twos:
        return "2"
    else:
        return "0"
    


In [440]:
# iterate over each neighbouring pair
aggr_rules = {}
for i in neighbours(variables):

    n1, n2 = i
    var, set_vars = diff_neighbours(n1, n2, variables)
    
    # go through list of dictionaries
    # collecting preferences from each profile
    preferences = []
    for cpnet in cp_list:
        
        # search for matching preferences
        prefs = matching_preferences(cpnet, set_vars, var)
        
        # TODO: what do we do, if we have conflicting preferences? is that possible?
        preferences= preferences + prefs
    
    # applying majority rule
    aggr_pref = maj_rule(preferences)  

    # build aggregated rule
    prem = frozenset(set_vars)
    if prem not in aggr_rules:
        aggr_rules[prem] = {reform_pref_string(var, aggr_pref)}        
    else:
        aggr_rules[prem].add(reform_pref_string(var, aggr_pref))

print(aggr_rules)   
        


{frozenset({'x5=1', 'x4=1', 'x2=1', 'x3=1'}): {'x1=0'}, frozenset({'x1=2', 'x4=1', 'x5=1', 'x3=1'}): {'x2=0'}, frozenset({'x1=2', 'x2=2', 'x4=1', 'x5=1'}): {'x3=0'}, frozenset({'x3=2', 'x2=2', 'x1=2', 'x5=1'}): {'x4=2'}, frozenset({'x3=2', 'x4=2', 'x1=2', 'x2=2'}): {'x5=2'}, frozenset({'x3=2', 'x2=2', 'x1=2', 'x4=1'}): {'x5=0'}, frozenset({'x3=2', 'x1=2', 'x2=2', 'x5=2'}): {'x4=2'}, frozenset({'x2=2', 'x1=2', 'x5=1', 'x3=1'}): {'x4=1'}, frozenset({'x2=2', 'x4=2', 'x1=2', 'x5=1'}): {'x3=0'}, frozenset({'x4=2', 'x1=2', 'x2=2', 'x3=1'}): {'x5=1'}, frozenset({'x4=2', 'x1=2', 'x2=2', 'x5=2'}): {'x3=0'}, frozenset({'x2=2', 'x1=2', 'x4=1', 'x3=1'}): {'x5=0'}, frozenset({'x2=2', 'x1=2', 'x4=1', 'x5=2'}): {'x3=0'}, frozenset({'x5=2', 'x1=2', 'x2=2', 'x3=1'}): {'x4=1'}, frozenset({'x5=1', 'x1=2', 'x4=1', 'x2=1'}): {'x3=1'}, frozenset({'x1=2', 'x3=2', 'x4=1', 'x5=1'}): {'x2=2'}, frozenset({'x5=1', 'x3=2', 'x1=2', 'x2=1'}): {'x4=2'}, frozenset({'x3=2', 'x4=2', 'x1=2', 'x5=1'}): {'x2=0'}, frozenset

In [441]:
# simplify the rule set
from itertools import combinations
for i in range(n-1, 0, -1):
    for subset in list(combinations(variables, i)):
        for i in neighbours(list(subset)):

            n1_orig, n2_orig = i
            var, set_vars_orig = diff_neighbours(n1_orig, n2_orig, subset)

            n1 = frozenset(set_vars_orig.union({var+"=1"})) 
            n2 = frozenset(set_vars_orig.union({var+"=2"}))

            set_vars = frozenset(set_vars_orig)

            if n1 in aggr_rules and n2 in aggr_rules:
                   
                intersection = aggr_rules[n1].intersection(aggr_rules[n2])
                diff_n1 = aggr_rules[n1].difference(aggr_rules[n2])
                diff_n2 = aggr_rules[n2].difference(aggr_rules[n1])

                if not not intersection:# elements are in the intersection
                    
                    aggr_rules.pop(n1)
                    aggr_rules.pop(n2)
                    
                    if set_vars not in aggr_rules:
                        aggr_rules[set_vars] = intersection       
                    else:
                        aggr_rules[set_vars].add(intersection.pop())
                    
                    if not not diff_n1: # TODO: funktioniert das?
                        aggr_rules[n1] = diff_n1
                    if not not diff_n2:
                        aggr_rules[n2] = diff_n2

print(aggr_rules)

{frozenset({'x3=2', 'x4=2', 'x2=2', 'x5=2'}): {'x1=2'}, frozenset({'x5=2', 'x3=2', 'x4=1', 'x1=1'}): {'x2=0'}, frozenset({'x5=1', 'x4=2', 'x2=1', 'x3=1'}): {'x1=0'}, frozenset({'x3=1', 'x4=2', 'x5=1', 'x1=1'}): {'x2=2'}, frozenset({'x5=2', 'x4=2', 'x2=1', 'x3=1'}): {'x1=1'}, frozenset({'x3=1', 'x5=2', 'x4=2', 'x1=1'}): {'x2=0'}, frozenset({'x5=2', 'x4=1', 'x2=1', 'x3=1'}): {'x1=0'}, frozenset({'x3=1', 'x5=2', 'x4=1', 'x1=1'}): {'x2=2'}, frozenset({'x1=2', 'x4=1', 'x3=1'}): {'x5=0'}, frozenset({'x3=1', 'x4=1', 'x1=1'}): {'x5=2'}, frozenset({'x3=2', 'x2=2', 'x4=1'}): {'x5=0', 'x1=2'}, frozenset({'x3=2', 'x4=2', 'x2=2'}): {'x5=2'}, frozenset({'x4=2', 'x2=2', 'x3=1'}): {'x5=1'}, frozenset({'x4=2', 'x2=1', 'x3=1'}): {'x5=0'}, frozenset({'x1=2', 'x2=2', 'x3=1'}): {'x4=1'}, frozenset({'x1=2', 'x2=1', 'x3=1'}): {'x4=0'}, frozenset({'x3=1', 'x2=1', 'x1=1'}): {'x4=2'}, frozenset({'x3=2', 'x1=2', 'x4=1'}): {'x2=2'}, frozenset({'x1=2', 'x5=1', 'x3=1'}): {'x2=0'}, frozenset({'x5=2', 'x1=2', 'x3=1'}

In [442]:
# measure number of rules 
count = 0 # elements in the dictionary have to be counted
for key in aggr_rules:
    count += len(aggr_rules[key])
print(count)

33


In [443]:
# measure number of indifferences
count = 0
for key in aggr_rules:
    for elem in aggr_rules[key]:
        if elem.endswith("0"):
            count += 1
print(count)

16


In [444]:
# measure the number of dependencies
dependencies = set()
for key in aggr_rules:
    for k_elem in key:
        for elem in aggr_rules[key]:
            dependencies.add((k_elem[:2], elem[:2]))
print(dependencies)            
print(len(dependencies))            

{('x3', 'x1'), ('x5', 'x1'), ('x1', 'x4'), ('x5', 'x2'), ('x3', 'x2'), ('x2', 'x4'), ('x3', 'x4'), ('x1', 'x2'), ('x4', 'x1'), ('x4', 'x2'), ('x3', 'x5'), ('x2', 'x5'), ('x2', 'x3'), ('x4', 'x5'), ('x1', 'x5'), ('x2', 'x1')}
16
